# MLflow 🔄

Este notebook contiene las técnicas de clasificación probadas para el TFM.

### **Ejecutar en línea de comandos:**

Para usar el interfaz de usuario se deben seguir los siguientes pasos:

1. Abrir la consola.

2. Ubicarse en el directorio donde se encuentra el archivo del código.

----------------------

Por ejemplo:

cd Desktop\TFM

---------------------

3. Ejecutar la siguiente expresión en la consola/línea de comandos:

mlflow ui

4. Tras ejecutar lo anterior, aparecerá un enlace que se deberá copiar y pegar en un buscador.
-------------------

Tendrá la siguiente forma: http://127.0.0.1:5000

-------------------

**NOTA:** Si has seguido todos los pasos y no aparecen los experimentos en la interfaz de usuario, es que te has ubicado en el directorio equivocado. Tendrás que repetir los pasos detallados.

Importamos librerías:

In [1]:
import numpy as np
import pandas as pd
import os
import sys
import time

import sklearn
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
# from tensorflow.keras.utils import to_categorical
# from tensorflow.keras import regularizers
# from keras import layers
# from tensorflow.keras import regularizers
# import tensorflow.keras as tk
# from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, RepeatedStratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
import keras
from keras import layers
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier

import mlflow
# Establecer la variable de entorno GIT_PYTHON_REFRESH en quiet
os.environ["GIT_PYTHON_REFRESH"] = "quiet"

Importamos los datos:

In [2]:
df_data = pd.read_csv('data.csv')
X = df_data.drop(columns=['battery','OHS','CHS', 'chemistry'])
y = df_data['OHS']

In [3]:
X.shape, y.shape

((118, 10), (118,))

# Decision Tree 🌳

In [4]:
experiment_tree = mlflow.create_experiment("tree")

In [5]:
experiment_tree

'762358564258105129'

In [6]:
with mlflow.start_run(experiment_id=experiment_tree) as run:
    seed = 10
    mlflow.log_param('seed', seed)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=seed)

    dt_classifier = DecisionTreeClassifier(random_state=seed)

    param_grid = {
        'max_depth': range(3, 11),  # Profundidad máxima del árbol
        'min_samples_split': range(2, 10),  # Número mínimo de muestras requeridas para dividir un nodo interno
        'min_samples_leaf': range(1, 8)  # Número mínimo de muestras requeridas para estar en un nodo hoja
    }
    
    mlflow.log_param('param_grid', param_grid)
    
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=seed)
    grid_search = GridSearchCV(estimator=dt_classifier, param_grid=param_grid,
                                     scoring='accuracy', cv=cv)
    
    mlflow.log_param('scoring', 'accuracy')
    mlflow.log_param('cv', cv)
    
    # Entrena el modelo
    start_time = time.time()
    grid_search.fit(X_train, y_train)
    train_time = time.time() - start_time
    
    # Obtiene los mejores parámetros
    best_params = grid_search.best_params_
    print("Mejores parámetros:", best_params)

    mlflow.log_param('best_params', best_params)
    
    # Predice usando el mejor modelo encontrado
    y_train_pred = grid_search.predict(X_train)
    y_test_pred = grid_search.predict(X_test)
    
    # Calcula la precisión del conjunto de entrenamiento y de prueba
    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    
    print("Precisión del conjunto de entrenamiento:", train_accuracy)
    print("Precisión del conjunto de prueba:", test_accuracy)

    mlflow.log_metric('train_time', train_time)
    mlflow.log_metric('train_accuracy', train_accuracy)
    mlflow.log_metric('test_accuracy', test_accuracy)

    # Convierte grid_search.cv_results_ en un DataFrame de pandas
    cv_results_df = pd.DataFrame(grid_search.cv_results_)
    
    # Guarda el DataFrame como un archivo CSV temporal
    temp_csv_file = "cv_results.csv"
    cv_results_df.to_csv(temp_csv_file, index=False)

    mlflow.sklearn.log_model(grid_search, 'model')
    
    print(f"Model path: runs:/{run.info.run_id}/model")


C:\Users\beca1\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\beca1\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\beca1\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


Mejores parámetros: {'max_depth': 4, 'min_samples_leaf': 4, 'min_samples_split': 2}
Precisión del conjunto de entrenamiento: 0.8409090909090909
Precisión del conjunto de prueba: 0.6333333333333333
Model path: runs:/a5981829bc5043bda3226d98e387ec60/model


C:\Users\beca1\AppData\Local\anaconda3\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [7]:
experiment = mlflow.get_experiment_by_name('tree')

In [8]:
experiment.experiment_id

'762358564258105129'

In [9]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import mlflow
import mlflow.sklearn

# Inicia un nuevo run de MLflow
with mlflow.start_run(experiment_id=experiment.experiment_id) as run:
    mlflow.log_param('seed-train_test_split', 0)
    
    # Divide los datos en conjuntos de entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)
    
    # Crea el modelo de árbol de clasificación
    mlflow.log_param('seed-DecisionTreeClassifier', 12)
    clf = DecisionTreeClassifier(random_state=12)
    
    # Entrena el modelo en los datos de entrenamiento
    clf.fit(X_train, y_train)
    
    # Realiza predicciones en los datos de prueba
    y_pred = clf.predict(X_test)
    
    # Calcula la precisión del modelo
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy:", accuracy)
    
    # Registra los parámetros y métricas en MLflow
    mlflow.log_param('test_size', 0.25)
    mlflow.log_param('seed', 0)
    mlflow.log_metric('test_accuracy', accuracy)
    
    # Guarda el modelo en MLflow
    mlflow.sklearn.log_model(clf, 'model')
    
    print(f"Model path: runs:/{run.info.run_id}/model")


Accuracy: 0.8
Model path: runs:/5748dff897274d3f9ec6b5543e915d92/model


# Random Forest 🌳🌲🌴🌳

In [10]:
experiment_rf = mlflow.create_experiment("rf")

In [11]:
experiment_rf = mlflow.get_experiment_by_name("rf")
print(experiment_rf)

<Experiment: artifact_location='file:///C:/Users/beca1/Documents/GitHub/PR-01062_ESA_INNOVATION_TFM_CH/COPY----------TFM_CODIGO/mlruns/897831684518675220', creation_time=1718262588810, experiment_id='897831684518675220', last_update_time=1718262588810, lifecycle_stage='active', name='rf', tags={}>


In [13]:
with mlflow.start_run(experiment_id=experiment_rf.experiment_id) as run:
    seed = 0
    mlflow.log_param('seed', seed)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=seed)

    rf_classifier = RandomForestClassifier(bootstrap=True, class_weight='balanced_subsample', oob_score=True, random_state=seed)

    mlflow.log_param('bootstrap', True)
    mlflow.log_param('class_weight', 'balanced_subsample')
    mlflow.log_param('oob_score', True)
    
    param_grid = {
        'n_estimators': range(30,250),  # Número de árboles en el bosque
        'max_depth': range(3,15),  # Profundidad máxima del árbol
        'min_samples_split': range(2,10),  # Número mínimo de muestras requeridas para dividir un nodo interno
        'min_samples_leaf': range(1,10),  # Número mínimo de muestras requeridas para estar en un nodo hoja
    }
    
    
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=seed)
    grid_search = RandomizedSearchCV(estimator=rf_classifier, param_distributions=param_grid,
                                     n_iter=100, scoring='accuracy', random_state=seed,
                                     cv=cv,) 

    mlflow.log_param('param_distributions', param_grid)
    mlflow.log_param('n_iter', 100)
    mlflow.log_param('scoring', 'accuracy')
    mlflow.log_param('cv', cv)
    
    # NOTA: SI cv=None => cv=5 por defecto.
    
    # Entrena el modelo
    start_time = time.time()
    grid_search.fit(X_train, y_train)
    train_time = time.time() - start_time
    
    # Obtiene los mejores parámetros
    best_params = grid_search.best_params_
    print("Mejores parámetros:", best_params)

    mlflow.log_param('cv', cv)
    mlflow.log_param('best_params', best_params)
    
    
    # Predice usando el mejor modelo encontrado
    y_train_pred = grid_search.predict(X_train)
    y_test_pred = grid_search.predict(X_test)
    
    # Calcula la precisión del conjunto de entrenamiento y de prueba
    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    
    print("Precisión del conjunto de entrenamiento:", train_accuracy)
    print("Precisión del conjunto de prueba:", test_accuracy)

    mlflow.log_metric('train_time', train_time)
    mlflow.log_metric('train_accuracy', train_accuracy)
    mlflow.log_metric('test_accuracy', test_accuracy)


    # Convierte grid_search.cv_results_ en un DataFrame de pandas
    cv_results_df = pd.DataFrame(grid_search.cv_results_)
    
    # Guarda el DataFrame como un archivo CSV temporal
    # Convierte grid_search.cv_results_ en un DataFrame de pandas
    cv_results_df = pd.DataFrame(grid_search.cv_results_)
    
    # Guarda el DataFrame como un archivo CSV temporal
    temp_csv_file = "cv_results.csv"
    cv_results_df.to_csv(temp_csv_file, index=False)

    mlflow.sklearn.log_model(grid_search, 'model')
    
    print(f"Model path: runs:/{run.info.run_id}/model")

C:\Users\beca1\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\beca1\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\beca1\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


Mejores parámetros: {'n_estimators': 30, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': 7}
Precisión del conjunto de entrenamiento: 0.9886363636363636
Precisión del conjunto de prueba: 0.7
Model path: runs:/9621344ffbb44d83aeadfe33935f3321/model


In [14]:
experiment_rf = mlflow.get_experiment_by_name("rf")
with mlflow.start_run(experiment_id=experiment_rf.experiment_id) as run:
    seed = 0
    mlflow.log_param('seed', seed)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=seed)

    rf_classifier = RandomForestClassifier(random_state=seed, bootstrap=True)

    # mlflow.log_param('bootstrap', True)
    # mlflow.log_param('class_weight', 'balanced_subsample')
    # mlflow.log_param('oob_score', True)
    
    param_grid = {
        'n_estimators': [50, 100, 200],  # Número de árboles en el bosque
        'max_depth': [2, 4, 6, None],  # Profundidad máxima del árbol
        'class_weight': ['balanced', 'balanced_subsample', None],
        'bootstrap': [True, False] 
    }
    
    
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=seed)
    grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid,
                                     scoring='accuracy', cv=cv) 

    mlflow.log_param('param_grid', param_grid)
    # mlflow.log_param('n_iter', 100)
    mlflow.log_param('scoring', 'accuracy')
    mlflow.log_param('cv', cv)
    
    # NOTA: SI cv=None => cv=5 por defecto.
    
    # Entrena el modelo
    start_time = time.time()
    grid_search.fit(X_train, y_train)
    train_time = time.time() - start_time
    
    # Obtiene los mejores parámetros
    best_params = grid_search.best_params_
    print("Mejores parámetros:", best_params)

    mlflow.log_param('cv', cv)
    mlflow.log_param('best_params', best_params)
    
    
    # Predice usando el mejor modelo encontrado
    y_train_pred = grid_search.predict(X_train)
    y_test_pred = grid_search.predict(X_test)
    
    # Calcula la precisión del conjunto de entrenamiento y de prueba
    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    
    print("Precisión del conjunto de entrenamiento:", train_accuracy)
    print("Precisión del conjunto de prueba:", test_accuracy)

    mlflow.log_metric('train_time', train_time)
    mlflow.log_metric('train_accuracy', train_accuracy)
    mlflow.log_metric('test_accuracy', test_accuracy)


    # Convierte grid_search.cv_results_ en un DataFrame de pandas
    cv_results_df = pd.DataFrame(grid_search.cv_results_)
    
    # Guarda el DataFrame como un archivo CSV temporal
    # Convierte grid_search.cv_results_ en un DataFrame de pandas
    cv_results_df = pd.DataFrame(grid_search.cv_results_)
    
    # Guarda el DataFrame como un archivo CSV temporal
    temp_csv_file = "cv_results.csv"
    cv_results_df.to_csv(temp_csv_file, index=False)

    mlflow.sklearn.log_model(grid_search, 'model')
    
    print(f"Model path: runs:/{run.info.run_id}/model")

C:\Users\beca1\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\beca1\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\beca1\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


Mejores parámetros: {'bootstrap': True, 'class_weight': 'balanced', 'max_depth': None, 'n_estimators': 100}
Precisión del conjunto de entrenamiento: 1.0
Precisión del conjunto de prueba: 0.8
Model path: runs:/0fa28b2cf78a493397f95f11929ee05a/model


In [15]:
experiment_rf = mlflow.get_experiment_by_name("rf")
with mlflow.start_run(experiment_id=experiment_rf.experiment_id) as run:
    seed = 0
    mlflow.log_param('seed', seed)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=seed)

    rf_classifier = RandomForestClassifier(bootstrap=True, 
                                           # class_weight='balanced_subsample', 
                                           oob_score=True, 
                                           random_state=seed)

    mlflow.log_param('bootstrap', True)
    # mlflow.log_param('class_weight', 'balanced_subsample')
    mlflow.log_param('oob_score', True)
    
    param_grid = {
        'n_estimators': range(30,250),  # Número de árboles en el bosque
        'max_depth': range(3,15),  # Profundidad máxima del árbol
        'min_samples_split': range(2,10),  # Número mínimo de muestras requeridas para dividir un nodo interno
        'min_samples_leaf': range(1,10),  # Número mínimo de muestras requeridas para estar en un nodo hoja
    }
    
    
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=seed)
    grid_search = RandomizedSearchCV(estimator=rf_classifier, param_distributions=param_grid,
                                     n_iter=100, scoring='accuracy', random_state=seed,
                                     cv=cv,) 

    mlflow.log_param('param_distributions', param_grid)
    mlflow.log_param('n_iter', 100)
    mlflow.log_param('scoring', 'accuracy')
    mlflow.log_param('cv', cv)
    
    # NOTA: SI cv=None => cv=5 por defecto.
    
    # Entrena el modelo
    start_time = time.time()
    grid_search.fit(X_train, y_train)
    train_time = time.time() - start_time
    
    # Obtiene los mejores parámetros
    best_params = grid_search.best_params_
    print("Mejores parámetros:", best_params)

    mlflow.log_param('cv', cv)
    mlflow.log_param('best_params', best_params)
    
    
    # Predice usando el mejor modelo encontrado
    y_train_pred = grid_search.predict(X_train)
    y_test_pred = grid_search.predict(X_test)
    
    # Calcula la precisión del conjunto de entrenamiento y de prueba
    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    
    print("Precisión del conjunto de entrenamiento:", train_accuracy)
    print("Precisión del conjunto de prueba:", test_accuracy)

    mlflow.log_metric('train_time', train_time)
    mlflow.log_metric('train_accuracy', train_accuracy)
    mlflow.log_metric('test_accuracy', test_accuracy)


    # Convierte grid_search.cv_results_ en un DataFrame de pandas
    cv_results_df = pd.DataFrame(grid_search.cv_results_)
    
    # Guarda el DataFrame como un archivo CSV temporal
    # Convierte grid_search.cv_results_ en un DataFrame de pandas
    cv_results_df = pd.DataFrame(grid_search.cv_results_)
    
    # Guarda el DataFrame como un archivo CSV temporal
    temp_csv_file = "cv_results.csv"
    cv_results_df.to_csv(temp_csv_file, index=False)

    mlflow.sklearn.log_model(grid_search, 'model')
    
    print(f"Model path: runs:/{run.info.run_id}/model")

C:\Users\beca1\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\beca1\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\beca1\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


Mejores parámetros: {'n_estimators': 113, 'min_samples_split': 6, 'min_samples_leaf': 1, 'max_depth': 8}
Precisión del conjunto de entrenamiento: 0.9545454545454546
Precisión del conjunto de prueba: 0.8
Model path: runs:/b06d311bb5724e3180239998e28a602d/model


In [16]:
experiment_rf = mlflow.get_experiment_by_name("rf")
with mlflow.start_run(experiment_id=experiment_rf.experiment_id) as run:
    seed = 0
    mlflow.log_param('seed', seed)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=seed)

    rf_classifier = RandomForestClassifier(bootstrap=True, 
                                           # class_weight='balanced_subsample', 
                                           oob_score=True, 
                                           random_state=seed)

    mlflow.log_param('bootstrap', True)
    # mlflow.log_param('class_weight', 'balanced_subsample')
    mlflow.log_param('oob_score', True)
    
    param_grid = {
        'n_estimators': range(30,250),  # Número de árboles en el bosque
        'max_depth': range(3,15),  # Profundidad máxima del árbol
        'min_samples_split': range(2,10),  # Número mínimo de muestras requeridas para dividir un nodo interno
        'min_samples_leaf': range(1,10),  # Número mínimo de muestras requeridas para estar en un nodo hoja
    }
    
    
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=seed)
    grid_search = RandomizedSearchCV(estimator=rf_classifier, param_distributions=param_grid,
                                     n_iter=100, scoring='accuracy', random_state=seed,
                                     cv=cv,) 

    mlflow.log_param('param_distributions', param_grid)
    mlflow.log_param('n_iter', 100)
    mlflow.log_param('scoring', 'accuracy')
    mlflow.log_param('cv', cv)
    
    # NOTA: SI cv=None => cv=5 por defecto.
    
    # Entrena el modelo
    start_time = time.time()
    grid_search.fit(X_train, y_train)
    train_time = time.time() - start_time
    
    # Obtiene los mejores parámetros
    best_params = grid_search.best_params_
    print("Mejores parámetros:", best_params)

    mlflow.log_param('cv', cv)
    mlflow.log_param('best_params', best_params)
    
    
    # Predice usando el mejor modelo encontrado
    y_train_pred = grid_search.predict(X_train)
    y_test_pred = grid_search.predict(X_test)
    
    # Calcula la precisión del conjunto de entrenamiento y de prueba
    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    
    print("Precisión del conjunto de entrenamiento:", train_accuracy)
    print("Precisión del conjunto de prueba:", test_accuracy)

    mlflow.log_metric('train_time', train_time)
    mlflow.log_metric('train_accuracy', train_accuracy)
    mlflow.log_metric('test_accuracy', test_accuracy)


    # Convierte grid_search.cv_results_ en un DataFrame de pandas
    cv_results_df = pd.DataFrame(grid_search.cv_results_)
    
    # Guarda el DataFrame como un archivo CSV temporal
    # Convierte grid_search.cv_results_ en un DataFrame de pandas
    cv_results_df = pd.DataFrame(grid_search.cv_results_)
    
    # Guarda el DataFrame como un archivo CSV temporal
    temp_csv_file = "cv_results.csv"
    cv_results_df.to_csv(temp_csv_file, index=False)

    mlflow.sklearn.log_model(grid_search, 'model')
    
    print(f"Model path: runs:/{run.info.run_id}/model")

C:\Users\beca1\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\beca1\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\beca1\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


Mejores parámetros: {'n_estimators': 113, 'min_samples_split': 6, 'min_samples_leaf': 1, 'max_depth': 8}
Precisión del conjunto de entrenamiento: 0.9545454545454546
Precisión del conjunto de prueba: 0.8
Model path: runs:/3c247b510ce949c09d339a601c8ed756/model


In [17]:
experiment_rf = mlflow.get_experiment_by_name("rf")
with mlflow.start_run(experiment_id=experiment_rf.experiment_id) as run:
    seed = 0
    mlflow.log_param('seed', seed)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=seed)

    rf_classifier = RandomForestClassifier(bootstrap=True, class_weight='balanced_subsample', oob_score=True, random_state=seed)

    mlflow.log_param('bootstrap', True)
    # mlflow.log_param('class_weight', 'balanced_subsample')
    mlflow.log_param('oob_score', True)
    
    param_grid = {
        'n_estimators': range(30,250),  # Número de árboles en el bosque
        'max_depth': range(3,15),  # Profundidad máxima del árbol
        'min_samples_split': range(2,10),  # Número mínimo de muestras requeridas para dividir un nodo interno
        'min_samples_leaf': range(1,10),  # Número mínimo de muestras requeridas para estar en un nodo hoja
    }
    
    
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=seed)
    grid_search = RandomizedSearchCV(estimator=rf_classifier, param_distributions=param_grid,
                                     n_iter=200, scoring='accuracy', random_state=seed,
                                     cv=cv,) 

    mlflow.log_param('param_distributions', param_grid)
    mlflow.log_param('n_iter', 200)
    mlflow.log_param('scoring', 'accuracy')
    mlflow.log_param('cv', cv)
    
    # NOTA: SI cv=None => cv=5 por defecto.
    
    # Entrena el modelo
    start_time = time.time()
    grid_search.fit(X_train, y_train)
    train_time = time.time() - start_time
    
    # Obtiene los mejores parámetros
    best_params = grid_search.best_params_
    print("Mejores parámetros:", best_params)

    mlflow.log_param('cv', cv)
    mlflow.log_param('best_params', best_params)
    
    
    # Predice usando el mejor modelo encontrado
    y_train_pred = grid_search.predict(X_train)
    y_test_pred = grid_search.predict(X_test)
    
    # Calcula la precisión del conjunto de entrenamiento y de prueba
    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    
    print("Precisión del conjunto de entrenamiento:", train_accuracy)
    print("Precisión del conjunto de prueba:", test_accuracy)

    mlflow.log_metric('train_time', train_time)
    mlflow.log_metric('train_accuracy', train_accuracy)
    mlflow.log_metric('test_accuracy', test_accuracy)


    # Convierte grid_search.cv_results_ en un DataFrame de pandas
    cv_results_df = pd.DataFrame(grid_search.cv_results_)
    
    # Guarda el DataFrame como un archivo CSV temporal
    # Convierte grid_search.cv_results_ en un DataFrame de pandas
    cv_results_df = pd.DataFrame(grid_search.cv_results_)
    
    # Guarda el DataFrame como un archivo CSV temporal
    temp_csv_file = "cv_results.csv"
    cv_results_df.to_csv(temp_csv_file, index=False)

    mlflow.sklearn.log_model(grid_search, 'model')
    
    print(f"Model path: runs:/{run.info.run_id}/model")

C:\Users\beca1\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\beca1\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\beca1\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


Mejores parámetros: {'n_estimators': 30, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': 7}
Precisión del conjunto de entrenamiento: 0.9886363636363636
Precisión del conjunto de prueba: 0.7
Model path: runs:/d65f78594ed84b849c656ee0343eb0c5/model


# $K$-NN 🏡🏡🏡

In [18]:
experiment_knn = mlflow.create_experiment("knn")

In [19]:
experiment_knn

'979228043064794755'

In [20]:
with mlflow.start_run(experiment_id=experiment_knn) as run:
    seed = 0
    mlflow.log_param('seed', seed)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=seed)
    
    # Define el pipeline con el escalado de características y el clasificador KNN
    pipeline = Pipeline([
        ('scaler', StandardScaler()),  # Escalado de características
        ('knn', KNeighborsClassifier())  # Clasificador KNN
    ])
    
    # Definir la cuadrícula de parámetros que quieres probar
    param_grid = {
        'knn__n_neighbors': [3, 4, 5, 6, 7, 8, 9],  # Prueba diferentes valores para el número de vecinos
        'knn__weights': ['uniform', 'distance'],  # Prueba diferentes métodos de ponderación
        'knn__metric': ['euclidean', 'manhattan', 'minkowski', 'chebyshev']  # Prueba diferentes métricas de distancia
    }

    mlflow.log_param('param_grid', param_grid)
    
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=seed)
    mlflow.log_param('cv', cv)
    
    # Instanciar el objeto GridSearchCV
    grid_search = GridSearchCV(pipeline, param_grid, cv=cv, scoring='accuracy')

    mlflow.log_param('scoring', 'accuracy')
    
    # Ajustar el objeto GridSearchCV a los datos de entrenamiento
    start_time = time.time()
    grid_search.fit(X_train, y_train)
    train_time = time.time() - start_time
    
    # Mostrar los mejores parámetros encontrados
    print("Mejores parámetros:", grid_search.best_params_)

    mlflow.log_param('best_params_', grid_search.best_params_)
    
    # Obtener el mejor modelo
    best_knn = grid_search.best_estimator_    
    
    # Hacer predicciones en el conjunto de prueba utilizando el mejor modelo
    y_train_pred = best_knn.predict(X_train)
    y_test_pred = best_knn.predict(X_test)
    
    # Calcular la precisión del modelo
    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    print("Precisión con los mejores parámetros:", test_accuracy)

    mlflow.log_metric('train_time', train_time)
    mlflow.log_metric('train_accuracy', train_accuracy)
    mlflow.log_metric('test_accuracy', test_accuracy)

    mlflow.sklearn.log_model(best_knn, 'model')
    
    print(f"Model path: runs:/{run.info.run_id}/model")


C:\Users\beca1\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\beca1\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\beca1\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


Mejores parámetros: {'knn__metric': 'euclidean', 'knn__n_neighbors': 9, 'knn__weights': 'distance'}
Precisión con los mejores parámetros: 0.8333333333333334
Model path: runs:/e2421c159f9a4fabada4d462f33c36b5/model


In [21]:
experiment_knn = mlflow.get_experiment_by_name("knn")
with mlflow.start_run(experiment_id=experiment_knn.experiment_id) as run:
    seed = 0
    mlflow.log_param('seed', seed)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=seed)
    # Definir la cuadrícula de parámetros que quieres probar
    param_grid = {
        'n_neighbors': [3, 4, 5, 6, 7, 8, 9],  # Prueba diferentes valores para el número de vecinos
        'weights': ['uniform', 'distance'],  # Prueba diferentes métodos de ponderación
        'metric': ['euclidean', 'manhattan', 'minkowski', 'chebyshev']  # Prueba diferentes métricas de distancia
    }

    mlflow.log_param('param_grid', param_grid)
    
    # Crear un clasificador KNN
    knn = KNeighborsClassifier()

    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=seed)
    mlflow.log_param('cv', cv)
    
    # Instanciar el objeto GridSearchCV
    grid_search = GridSearchCV(knn, param_grid, cv=cv, scoring='accuracy', )

    mlflow.log_param('scoring', 'accuracy')
    
    # Ajustar el objeto GridSearchCV a los datos de entrenamiento
    start_time = time.time()
    grid_search.fit(X_train, y_train)
    train_time = time.time() - start_time
    
    # Mostrar los mejores parámetros encontrados
    print("Mejores parámetros:", grid_search.best_params_)

    mlflow.log_param('best_params_', grid_search.best_params_)
    
    # Obtener el mejor modelo
    best_knn = grid_search.best_estimator_    
    
    # Hacer predicciones en el conjunto de prueba utilizando el mejor modelo
    y_train_pred = best_knn.predict(X_train)
    y_test_pred = best_knn.predict(X_test)
    
    # Calcular la precisión del modelo
    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    print("Precisión con los mejores parámetros:", test_accuracy)

    mlflow.log_metric('train_time', train_time)
    mlflow.log_metric('train_accuracy', train_accuracy)
    mlflow.log_metric('test_accuracy', test_accuracy)

    mlflow.sklearn.log_model(best_knn, 'model')
    
    print(f"Model path: runs:/{run.info.run_id}/model")


C:\Users\beca1\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\beca1\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\beca1\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


Mejores parámetros: {'metric': 'manhattan', 'n_neighbors': 3, 'weights': 'distance'}
Precisión con los mejores parámetros: 0.8333333333333334
Model path: runs:/d119bafba2204a338d93bf37bf9c1adb/model


# NN 🧬

In [22]:
experiment_nn = mlflow.create_experiment("nn")

In [23]:
experiment_nn = mlflow.get_experiment_by_name("nn")

In [24]:
experiment_nn

<Experiment: artifact_location='file:///C:/Users/beca1/Documents/GitHub/PR-01062_ESA_INNOVATION_TFM_CH/COPY----------TFM_CODIGO/mlruns/259948346618525345', creation_time=1718264959573, experiment_id='259948346618525345', last_update_time=1718264959573, lifecycle_stage='active', name='nn', tags={}>

In [26]:
# Inicia un nuevo run de MLflow
with mlflow.start_run(experiment_id=experiment_nn.experiment_id) as run:
    seed = 0
    mlflow.log_param('seed', seed)

    # Codifica las etiquetas a valores numéricos si aún no están codificadas
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
        
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=seed)

    # Define el modelo de la red neuronal
    model = Sequential()
    model.add(Dense(64, input_dim=10, activation='relu'))  # Capa de entrada con 10 variables de entrada
    model.add(Dense(32, activation='relu'))  # Capa oculta con 32 neuronas
    model.add(Dense(32, activation='relu'))  # Capa oculta con 32 neuronas
    model.add(Dense(8, activation='softmax'))  # Capa de salida con 8 neuronas para las 8 categorías

    # Compila el modelo
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Entrena el modelo
    start_time = time.time()
    history = model.fit(X_train, y_train, epochs=60, batch_size=8, validation_data=(X_test, y_test))
    train_time = time.time() - start_time

    # Evalúa el modelo en el conjunto de prueba
    loss, accuracy = model.evaluate(X_test, y_test)
    print("Accuracy del modelo en el conjunto de prueba:", accuracy)
    
    # Log de los parámetros y métricas en MLflow
    mlflow.log_param('batch_size', 8)
    mlflow.log_param('epochs', 50)
    mlflow.log_metric('test_loss', loss)
    mlflow.log_metric('test_accuracy', accuracy)
    mlflow.log_metric('train_time', train_time)
    

    # Guarda el modelo en MLflow
    mlflow.keras.log_model(model, "model")
    print(f"Model path: runs:/{run.info.run_id}/model")




Epoch 1/60


12/12 [==============================] - 2s 23ms/step - loss: 782.9533 - accuracy: 0.3085 - val_loss: 657.7432 - val_accuracy: 0.1667
Epoch 2/60
12/12 [==============================] - 0s 4ms/step - loss: 389.2849 - accuracy: 0.2234 - val_loss: 279.2611 - val_accuracy: 0.2083
Epoch 3/60
12/12 [==============================] - 0s 8ms/step - loss: 111.1857 - accuracy: 0.2234 - val_loss: 103.2326 - val_accuracy: 0.1250
Epoch 4/60
12/12 [==============================] - 0s 4ms/step - loss: 49.1210 - accuracy: 0.2128 - val_loss: 75.5264 - val_accuracy: 0.3750
Epoch 5/60
12/12 [==============================] - 0s 4ms/step - loss: 36.3477 - accuracy: 0.2128 - val_loss: 56.5016 - val_accuracy: 0.2083
Epoch 6/60
12/12 [==============================] - 0s 4ms/step - loss: 29.9403 - accuracy: 0.2234 - val_loss: 51.2043 - val_accuracy: 0.5833
Epoch 7/60
12/12 [==============================] - 0s 4ms/step - loss: 25.5647 - accuracy: 0.2021 - val_loss: 47.7847 - val_accuracy: 0.

2024/06/13 09:55:01 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


Accuracy del modelo en el conjunto de prueba: 0.5
INFO:tensorflow:Assets written to: C:\Users\beca1\AppData\Local\Temp\tmp8bd6bgw_\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\beca1\AppData\Local\Temp\tmp8bd6bgw_\model\data\model\assets


Model path: runs:/039ff494d1c647b3ac84e4f6c225930d/model


In [27]:
# Inicia un nuevo run de MLflow
with mlflow.start_run(experiment_id=experiment_nn.experiment_id) as run:
    seed = 0
    mlflow.log_param('seed', seed)

    # Codifica las etiquetas a valores numéricos si aún no están codificadas
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
        
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=seed)

    # Define el modelo de la red neuronal
    model = Sequential()
    model.add(Dense(64, input_dim=10, activation='relu'))  # Capa de entrada con 10 variables de entrada
    model.add(Dense(32, activation='relu'))  # Capa oculta con 32 neuronas
    model.add(Dense(32, activation='relu'))  # Capa oculta con 32 neuronas
    model.add(Dense(8, activation='softmax'))  # Capa de salida con 8 neuronas para las 8 categorías

    # Compila el modelo
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Entrena el modelo
    start_time = time.time()
    history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_test, y_test))
    train_time = time.time() - start_time

    # Evalúa el modelo en el conjunto de prueba
    loss, accuracy = model.evaluate(X_test, y_test)
    print("Accuracy del modelo en el conjunto de prueba:", accuracy)
    
    # Log de los parámetros y métricas en MLflow
    mlflow.log_param('batch_size', 8)
    mlflow.log_param('epochs', 50)
    mlflow.log_metric('test_loss', loss)
    mlflow.log_metric('test_accuracy', accuracy)
    mlflow.log_metric('train_time', train_time)
    

    # Guarda el modelo en MLflow
    mlflow.keras.log_model(model, "model")
    print(f"Model path: runs:/{run.info.run_id}/model")


Epoch 1/50
6/6 [==============================] - 1s 42ms/step - loss: 125.0975 - accuracy: 0.3085 - val_loss: 47.8311 - val_accuracy: 0.2083
Epoch 2/50
6/6 [==============================] - 0s 8ms/step - loss: 67.4601 - accuracy: 0.2979 - val_loss: 25.6145 - val_accuracy: 0.2500
Epoch 3/50
6/6 [==============================] - 0s 7ms/step - loss: 40.1006 - accuracy: 0.2660 - val_loss: 12.0441 - val_accuracy: 0.3750
Epoch 4/50
6/6 [==============================] - 0s 15ms/step - loss: 38.2156 - accuracy: 0.3191 - val_loss: 7.3191 - val_accuracy: 0.5833
Epoch 5/50
6/6 [==============================] - 0s 8ms/step - loss: 33.0908 - accuracy: 0.3936 - val_loss: 7.4377 - val_accuracy: 0.2500
Epoch 6/50
6/6 [==============================] - 0s 6ms/step - loss: 31.6531 - accuracy: 0.4043 - val_loss: 11.9029 - val_accuracy: 0.3333
Epoch 7/50
6/6 [==============================] - 0s 7ms/step - loss: 27.7751 - accuracy: 0.4149 - val_loss: 4.3713 - val_accuracy: 0.5000
Epoch 8/50
6/6 [====

2024/06/13 09:55:29 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\beca1\AppData\Local\Temp\tmpkcy5tfcg\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\beca1\AppData\Local\Temp\tmpkcy5tfcg\model\data\model\assets


Model path: runs:/98720ad0aff740ae9af7c73ddd2a3f4d/model


In [28]:
# Inicia un nuevo run de MLflow
with mlflow.start_run(experiment_id=experiment_nn.experiment_id) as run:
    seed = 0
    mlflow.log_param('seed', seed)

    # Codifica las etiquetas a valores numéricos si aún no están codificadas
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
        
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=seed)

    # Define el modelo de la red neuronal
    model = Sequential()
    model.add(Dense(64, input_dim=10, activation='relu'))  # Capa de entrada con 10 variables de entrada
    model.add(Dense(32, activation='relu'))  # Capa oculta con 32 neuronas
    model.add(Dense(32, activation='relu'))  # Capa oculta con 32 neuronas
    model.add(Dense(8, activation='softmax'))  # Capa de salida con 8 neuronas para las 8 categorías

    # Compila el modelo
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Entrena el modelo
    start_time = time.time()
    history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))
    train_time = time.time() - start_time

    # Evalúa el modelo en el conjunto de prueba
    loss, accuracy = model.evaluate(X_test, y_test)
    print("Accuracy del modelo en el conjunto de prueba:", accuracy)
    
    # Log de los parámetros y métricas en MLflow
    mlflow.log_param('batch_size', 8)
    mlflow.log_param('epochs', 50)
    mlflow.log_metric('test_loss', loss)
    mlflow.log_metric('test_accuracy', accuracy)
    mlflow.log_metric('train_time', train_time)
    

    # Guarda el modelo en MLflow
    mlflow.keras.log_model(model, "model")
    print(f"Model path: runs:/{run.info.run_id}/model")


Epoch 1/100
3/3 [==============================] - 1s 97ms/step - loss: 431.0324 - accuracy: 0.0745 - val_loss: 342.4226 - val_accuracy: 0.1250
Epoch 2/100
3/3 [==============================] - 0s 15ms/step - loss: 311.4723 - accuracy: 0.1383 - val_loss: 180.8130 - val_accuracy: 0.0417
Epoch 3/100
3/3 [==============================] - 0s 16ms/step - loss: 193.6108 - accuracy: 0.0319 - val_loss: 103.6400 - val_accuracy: 0.0000e+00
Epoch 4/100
3/3 [==============================] - 0s 14ms/step - loss: 119.7700 - accuracy: 0.2234 - val_loss: 66.9099 - val_accuracy: 0.4583
Epoch 5/100
3/3 [==============================] - 0s 14ms/step - loss: 87.7727 - accuracy: 0.3511 - val_loss: 48.5866 - val_accuracy: 0.4167
Epoch 6/100
3/3 [==============================] - 0s 17ms/step - loss: 72.0769 - accuracy: 0.3511 - val_loss: 71.5027 - val_accuracy: 0.2500
Epoch 7/100
3/3 [==============================] - 0s 17ms/step - loss: 77.2479 - accuracy: 0.2872 - val_loss: 51.8274 - val_accuracy: 0.

2024/06/13 09:55:48 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


Accuracy del modelo en el conjunto de prueba: 0.1666666716337204
INFO:tensorflow:Assets written to: C:\Users\beca1\AppData\Local\Temp\tmpe37ypmd1\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\beca1\AppData\Local\Temp\tmpe37ypmd1\model\data\model\assets


Model path: runs:/630eb37ded1849f6acac0550d708f559/model


In [29]:
# Inicia un nuevo run de MLflow
with mlflow.start_run(experiment_id=experiment_nn.experiment_id) as run:
    seed = 0
    mlflow.log_param('seed', seed)

    # Codifica las etiquetas a valores numéricos si aún no están codificadas
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
        
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=seed)

    # Define el modelo de la red neuronal
    model = Sequential()
    model.add(Dense(64, input_dim=10, activation='relu'))  # Capa de entrada con 10 variables de entrada
    model.add(Dense(32, activation='relu'))  # Capa oculta con 32 neuronas
    model.add(Dense(32, activation='relu'))  # Capa oculta con 32 neuronas
    model.add(Dense(8, activation='softmax'))  # Capa de salida con 8 neuronas para las 8 categorías

    # Compila el modelo
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Entrena el modelo
    start_time = time.time()
    history = model.fit(X_train, y_train, epochs=100, batch_size=24, validation_data=(X_test, y_test))
    train_time = time.time() - start_time

    # Evalúa el modelo en el conjunto de prueba
    loss, accuracy = model.evaluate(X_test, y_test)
    print("Accuracy del modelo en el conjunto de prueba:", accuracy)
    
    # Log de los parámetros y métricas en MLflow
    mlflow.log_param('batch_size', 8)
    mlflow.log_param('epochs', 50)
    mlflow.log_metric('test_loss', loss)
    mlflow.log_metric('test_accuracy', accuracy)
    mlflow.log_metric('train_time', train_time)
    

    # Guarda el modelo en MLflow
    mlflow.keras.log_model(model, "model")
    print(f"Model path: runs:/{run.info.run_id}/model")


Epoch 1/100
4/4 [==============================] - 1s 65ms/step - loss: 179.2078 - accuracy: 0.1277 - val_loss: 108.6193 - val_accuracy: 0.1667
Epoch 2/100
4/4 [==============================] - 0s 9ms/step - loss: 76.6915 - accuracy: 0.1596 - val_loss: 28.0107 - val_accuracy: 0.1667
Epoch 3/100
4/4 [==============================] - 0s 9ms/step - loss: 34.1017 - accuracy: 0.1277 - val_loss: 32.6218 - val_accuracy: 0.1250
Epoch 4/100
4/4 [==============================] - 0s 9ms/step - loss: 27.8598 - accuracy: 0.1064 - val_loss: 20.8099 - val_accuracy: 0.0833
Epoch 5/100
4/4 [==============================] - 0s 9ms/step - loss: 24.3076 - accuracy: 0.0957 - val_loss: 16.7157 - val_accuracy: 0.2917
Epoch 6/100
4/4 [==============================] - 0s 10ms/step - loss: 23.2647 - accuracy: 0.1915 - val_loss: 19.5871 - val_accuracy: 0.2083
Epoch 7/100
4/4 [==============================] - 0s 11ms/step - loss: 18.5902 - accuracy: 0.1277 - val_loss: 14.0870 - val_accuracy: 0.2083
Epoch 8/

2024/06/13 09:56:11 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


Accuracy del modelo en el conjunto de prueba: 0.7083333134651184
INFO:tensorflow:Assets written to: C:\Users\beca1\AppData\Local\Temp\tmpagrif6ly\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\beca1\AppData\Local\Temp\tmpagrif6ly\model\data\model\assets


Model path: runs:/b45bf92e755b4ecab2fab88a3f4bc36b/model


In [30]:
import mlflow
from mlflow.keras import log_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten
import time

# Inicia un nuevo run de MLflow
with mlflow.start_run(experiment_id=experiment_nn.experiment_id) as run:
    seed = 0
    mlflow.log_param('seed', seed)

    # Codifica las etiquetas a valores numéricos si aún no están codificadas
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
        
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=seed)

    # Define el modelo de la red neuronal convolucional
    model = Sequential()
    model.add(Conv1D(64, 3, activation='relu', input_shape=(10, 1)))  # Capa convolucional
    model.add(MaxPooling1D(2))  # Capa de pooling
    model.add(Flatten())  # Capa de aplanamiento
    model.add(Dense(32, activation='relu'))  # Capa oculta con 32 neuronas
    model.add(Dense(8, activation='softmax'))  # Capa de salida con 8 neuronas para las 8 categorías

    # Compila el modelo
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Entrena el modelo
    start_time = time.time()
    history = model.fit(X_train, y_train, epochs=100, batch_size=24, validation_data=(X_test, y_test))
    train_time = time.time() - start_time

    # Evalúa el modelo en el conjunto de prueba
    loss, accuracy = model.evaluate(X_test, y_test)
    print("Accuracy del modelo en el conjunto de prueba:", accuracy)
    
    # Log de los parámetros y métricas en MLflow
    mlflow.log_param('batch_size', 24)
    mlflow.log_param('epochs', 100)
    mlflow.log_metric('test_loss', loss)
    mlflow.log_metric('test_accuracy', accuracy)
    mlflow.log_metric('train_time', train_time)
    

    # Guarda el modelo en MLflow
    log_model(model, "model")
    print(f"Model path: runs:/{run.info.run_id}/model")


Epoch 1/100
4/4 [==============================] - 1s 69ms/step - loss: 255.4047 - accuracy: 0.1170 - val_loss: 219.1761 - val_accuracy: 0.3333
Epoch 2/100
4/4 [==============================] - 0s 13ms/step - loss: 175.0412 - accuracy: 0.2234 - val_loss: 182.0367 - val_accuracy: 0.1667
Epoch 3/100
4/4 [==============================] - 0s 10ms/step - loss: 130.7816 - accuracy: 0.2766 - val_loss: 141.3073 - val_accuracy: 0.1250
Epoch 4/100
4/4 [==============================] - 0s 11ms/step - loss: 101.4952 - accuracy: 0.2447 - val_loss: 113.9571 - val_accuracy: 0.0833
Epoch 5/100
4/4 [==============================] - 0s 11ms/step - loss: 78.5377 - accuracy: 0.2766 - val_loss: 82.5633 - val_accuracy: 0.1667
Epoch 6/100
4/4 [==============================] - 0s 13ms/step - loss: 50.3381 - accuracy: 0.3191 - val_loss: 53.8043 - val_accuracy: 0.3333
Epoch 7/100
4/4 [==============================] - 0s 10ms/step - loss: 29.9997 - accuracy: 0.3617 - val_loss: 50.5501 - val_accuracy: 0.250

2024/06/13 09:56:29 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


Accuracy del modelo en el conjunto de prueba: 0.6666666865348816
INFO:tensorflow:Assets written to: C:\Users\beca1\AppData\Local\Temp\tmp4ltxneco\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\beca1\AppData\Local\Temp\tmp4ltxneco\model\data\model\assets


Model path: runs:/7592a8aa0fa446d88aca533573098abb/model


In [31]:
import mlflow
from mlflow.keras import log_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten
import time

# Inicia un nuevo run de MLflow
with mlflow.start_run(experiment_id=experiment_nn.experiment_id) as run:
    seed = 0
    mlflow.log_param('seed', seed)

    # Codifica las etiquetas a valores numéricos si aún no están codificadas
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
        
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=seed)

    # Define el modelo de la red neuronal convolucional
    model = Sequential()
    model.add(Conv1D(64, 3, activation='relu', input_shape=(10, 1)))  # Capa convolucional
    model.add(MaxPooling1D(2))  # Capa de pooling
    model.add(Flatten())  # Capa de aplanamiento
    model.add(Dense(32, activation='relu'))  # Capa oculta con 32 neuronas
    model.add(Dense(8, activation='softmax'))  # Capa de salida con 8 neuronas para las 8 categorías

    # Compila el modelo
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Entrena el modelo
    start_time = time.time()
    history = model.fit(X_train, y_train, epochs=100, batch_size=24, validation_data=(X_test, y_test))
    train_time = time.time() - start_time

    # Evalúa el modelo en el conjunto de prueba
    loss, accuracy = model.evaluate(X_test, y_test)
    print("Accuracy del modelo en el conjunto de prueba:", accuracy)
    
    # Log de los parámetros y métricas en MLflow
    mlflow.log_param('batch_size', 24)
    mlflow.log_param('epochs', 100)
    mlflow.log_metric('test_loss', loss)
    mlflow.log_metric('test_accuracy', accuracy)
    mlflow.log_metric('train_time', train_time)
    

    # Guarda el modelo en MLflow
    log_model(model, "model")
    print(f"Model path: runs:/{run.info.run_id}/model")


Epoch 1/100
4/4 [==============================] - 1s 77ms/step - loss: 281.5653 - accuracy: 0.1489 - val_loss: 218.1594 - val_accuracy: 0.3333
Epoch 2/100
4/4 [==============================] - 0s 10ms/step - loss: 177.0300 - accuracy: 0.1170 - val_loss: 133.7119 - val_accuracy: 0.0833
Epoch 3/100
4/4 [==============================] - 0s 13ms/step - loss: 114.9297 - accuracy: 0.1915 - val_loss: 67.8341 - val_accuracy: 0.1667
Epoch 4/100
4/4 [==============================] - 0s 12ms/step - loss: 50.6155 - accuracy: 0.1170 - val_loss: 35.2426 - val_accuracy: 0.1250
Epoch 5/100
4/4 [==============================] - 0s 10ms/step - loss: 42.5863 - accuracy: 0.1702 - val_loss: 36.0262 - val_accuracy: 0.2083
Epoch 6/100
4/4 [==============================] - 0s 9ms/step - loss: 37.4828 - accuracy: 0.1809 - val_loss: 24.0057 - val_accuracy: 0.1250
Epoch 7/100
4/4 [==============================] - 0s 14ms/step - loss: 20.5402 - accuracy: 0.1277 - val_loss: 36.6741 - val_accuracy: 0.1667
Ep

2024/06/13 09:56:47 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


Accuracy del modelo en el conjunto de prueba: 0.5833333134651184
INFO:tensorflow:Assets written to: C:\Users\beca1\AppData\Local\Temp\tmp1ahdv9iu\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\beca1\AppData\Local\Temp\tmp1ahdv9iu\model\data\model\assets


Model path: runs:/aa6f1f87ccd14efba39c7b0170efd9b0/model


In [32]:
import mlflow
from mlflow.keras import log_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten
import time

# Inicia un nuevo run de MLflow
with mlflow.start_run(experiment_id=experiment_nn.experiment_id) as run:
    seed = 0
    mlflow.log_param('seed', seed)

    # Codifica las etiquetas a valores numéricos si aún no están codificadas
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
        
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=seed)

    # Define el modelo de la red neuronal convolucional
    model = Sequential()
    model.add(Conv1D(64, 3, activation='relu', input_shape=(10, 1)))  # Capa convolucional
    model.add(MaxPooling1D(2))  # Capa de pooling
    model.add(Flatten())  # Capa de aplanamiento
    model.add(Dense(32, activation='relu'))  # Capa oculta con 32 neuronas
    model.add(Dense(8, activation='softmax'))  # Capa de salida con 8 neuronas para las 8 categorías

    # Compila el modelo
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Entrena el modelo
    start_time = time.time()
    history = model.fit(X_train, y_train, epochs=200, batch_size=24, validation_data=(X_test, y_test))
    train_time = time.time() - start_time

    # Evalúa el modelo en el conjunto de prueba
    loss, accuracy = model.evaluate(X_test, y_test)
    print("Accuracy del modelo en el conjunto de prueba:", accuracy)
    
    # Log de los parámetros y métricas en MLflow
    mlflow.log_param('batch_size', 24)
    mlflow.log_param('epochs', 100)
    mlflow.log_metric('test_loss', loss)
    mlflow.log_metric('test_accuracy', accuracy)
    mlflow.log_metric('train_time', train_time)
    

    # Guarda el modelo en MLflow
    log_model(model, "model")
    print(f"Model path: runs:/{run.info.run_id}/model")


Epoch 1/200
4/4 [==============================] - 1s 77ms/step - loss: 408.6285 - accuracy: 0.0426 - val_loss: 217.8426 - val_accuracy: 0.0833
Epoch 2/200
4/4 [==============================] - 0s 10ms/step - loss: 268.7596 - accuracy: 0.0426 - val_loss: 102.5495 - val_accuracy: 0.0417
Epoch 3/200
4/4 [==============================] - 0s 9ms/step - loss: 156.8873 - accuracy: 0.1064 - val_loss: 115.8301 - val_accuracy: 0.1667
Epoch 4/200
4/4 [==============================] - 0s 9ms/step - loss: 129.8292 - accuracy: 0.1915 - val_loss: 70.7194 - val_accuracy: 0.0000e+00
Epoch 5/200
4/4 [==============================] - 0s 9ms/step - loss: 97.9920 - accuracy: 0.0213 - val_loss: 62.4899 - val_accuracy: 0.2083
Epoch 6/200
4/4 [==============================] - 0s 10ms/step - loss: 90.5123 - accuracy: 0.1170 - val_loss: 42.1488 - val_accuracy: 0.2500
Epoch 7/200
4/4 [==============================] - 0s 11ms/step - loss: 59.2616 - accuracy: 0.0745 - val_loss: 38.5374 - val_accuracy: 0.125

2024/06/13 09:57:09 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


Accuracy del modelo en el conjunto de prueba: 0.5833333134651184
INFO:tensorflow:Assets written to: C:\Users\beca1\AppData\Local\Temp\tmpbc1kpodq\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\beca1\AppData\Local\Temp\tmpbc1kpodq\model\data\model\assets


Model path: runs:/431d6761848d4a2686c32a609bf771cd/model


In [33]:
import mlflow
from mlflow.keras import log_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten
import time

# Inicia un nuevo run de MLflow
with mlflow.start_run(experiment_id=experiment_nn.experiment_id) as run:
    seed = 0
    mlflow.log_param('seed', seed)

    # Codifica las etiquetas a valores numéricos si aún no están codificadas
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
        
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=seed)

    # Define el modelo de la red neuronal convolucional
    model = Sequential()
    model.add(Conv1D(64, 3, activation='relu', input_shape=(10, 1)))  # Capa convolucional
    model.add(MaxPooling1D(2))  # Capa de pooling
    model.add(Flatten())  # Capa de aplanamiento
    model.add(Dense(32, activation='relu'))  # Capa oculta con 32 neuronas
    model.add(Dense(8, activation='softmax'))  # Capa de salida con 8 neuronas para las 8 categorías

    # Compila el modelo
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Entrena el modelo
    start_time = time.time()
    history = model.fit(X_train, y_train, epochs=500, batch_size=24, validation_data=(X_test, y_test))
    train_time = time.time() - start_time

    # Evalúa el modelo en el conjunto de prueba
    loss, accuracy = model.evaluate(X_test, y_test)
    print("Accuracy del modelo en el conjunto de prueba:", accuracy)
    
    # Log de los parámetros y métricas en MLflow
    mlflow.log_param('batch_size', 24)
    mlflow.log_param('epochs', 100)
    mlflow.log_metric('test_loss', loss)
    mlflow.log_metric('test_accuracy', accuracy)
    mlflow.log_metric('train_time', train_time)
    

    # Guarda el modelo en MLflow
    log_model(model, "model")
    print(f"Model path: runs:/{run.info.run_id}/model")


Epoch 1/500
4/4 [==============================] - 1s 66ms/step - loss: 167.4103 - accuracy: 0.1596 - val_loss: 81.6243 - val_accuracy: 0.4167
Epoch 2/500
4/4 [==============================] - 0s 9ms/step - loss: 103.6231 - accuracy: 0.2979 - val_loss: 71.9713 - val_accuracy: 0.1250
Epoch 3/500
4/4 [==============================] - 0s 9ms/step - loss: 79.6612 - accuracy: 0.2872 - val_loss: 42.1833 - val_accuracy: 0.3333
Epoch 4/500
4/4 [==============================] - 0s 9ms/step - loss: 61.2136 - accuracy: 0.3723 - val_loss: 28.7166 - val_accuracy: 0.5417
Epoch 5/500
4/4 [==============================] - 0s 10ms/step - loss: 46.0227 - accuracy: 0.4149 - val_loss: 25.6890 - val_accuracy: 0.2917
Epoch 6/500
4/4 [==============================] - 0s 10ms/step - loss: 34.8178 - accuracy: 0.2660 - val_loss: 12.1633 - val_accuracy: 0.2083
Epoch 7/500
4/4 [==============================] - 0s 11ms/step - loss: 20.3970 - accuracy: 0.2660 - val_loss: 3.7210 - val_accuracy: 0.5000
Epoch 8/

2024/06/13 09:57:40 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


Accuracy del modelo en el conjunto de prueba: 0.6666666865348816
INFO:tensorflow:Assets written to: C:\Users\beca1\AppData\Local\Temp\tmp7feaxthz\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\beca1\AppData\Local\Temp\tmp7feaxthz\model\data\model\assets


Model path: runs:/7e31680e6a8f45ed841fb2b321ef2a85/model


In [34]:
import mlflow
from mlflow.keras import log_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten
import time

# Inicia un nuevo run de MLflow
with mlflow.start_run(experiment_id=experiment_nn.experiment_id) as run:
    seed = 0
    mlflow.log_param('seed', seed)

    # Codifica las etiquetas a valores numéricos si aún no están codificadas
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
        
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=seed)

    # Define el modelo de la red neuronal convolucional
    model = Sequential()
    model.add(Conv1D(64, 3, activation='relu', input_shape=(10, 1)))  # Capa convolucional
    model.add(MaxPooling1D(2))  # Capa de pooling
    model.add(Flatten())  # Capa de aplanamiento
    model.add(Dense(32, activation='relu'))  # Capa oculta con 32 neuronas
    model.add(Dense(8, activation='softmax'))  # Capa de salida con 8 neuronas para las 8 categorías

    # Compila el modelo
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Entrena el modelo
    start_time = time.time()
    history = model.fit(X_train, y_train, epochs=1000, batch_size=24, validation_data=(X_test, y_test))
    train_time = time.time() - start_time

    # Evalúa el modelo en el conjunto de prueba
    loss, accuracy = model.evaluate(X_test, y_test)
    print("Accuracy del modelo en el conjunto de prueba:", accuracy)
    
    # Log de los parámetros y métricas en MLflow
    mlflow.log_param('batch_size', 24)
    mlflow.log_param('epochs', 100)
    mlflow.log_metric('test_loss', loss)
    mlflow.log_metric('test_accuracy', accuracy)
    mlflow.log_metric('train_time', train_time)
    

    # Guarda el modelo en MLflow
    log_model(model, "model")
    print(f"Model path: runs:/{run.info.run_id}/model")


Epoch 1/1000
4/4 [==============================] - 1s 71ms/step - loss: 214.6961 - accuracy: 0.1702 - val_loss: 91.0232 - val_accuracy: 0.4167
Epoch 2/1000
4/4 [==============================] - 0s 10ms/step - loss: 113.6344 - accuracy: 0.3617 - val_loss: 70.1184 - val_accuracy: 0.1667
Epoch 3/1000
4/4 [==============================] - 0s 10ms/step - loss: 106.1104 - accuracy: 0.2021 - val_loss: 49.0096 - val_accuracy: 0.2083
Epoch 4/1000
4/4 [==============================] - 0s 9ms/step - loss: 66.5814 - accuracy: 0.1809 - val_loss: 12.9417 - val_accuracy: 0.2500
Epoch 5/1000
4/4 [==============================] - 0s 9ms/step - loss: 26.7902 - accuracy: 0.4468 - val_loss: 26.6061 - val_accuracy: 0.4167
Epoch 6/1000
4/4 [==============================] - 0s 11ms/step - loss: 31.6546 - accuracy: 0.4468 - val_loss: 23.6958 - val_accuracy: 0.4583
Epoch 7/1000
4/4 [==============================] - 0s 12ms/step - loss: 24.5347 - accuracy: 0.3298 - val_loss: 25.4364 - val_accuracy: 0.250

2024/06/13 09:58:29 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


Accuracy del modelo en el conjunto de prueba: 0.7916666865348816
INFO:tensorflow:Assets written to: C:\Users\beca1\AppData\Local\Temp\tmpmcjrwrjx\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\beca1\AppData\Local\Temp\tmpmcjrwrjx\model\data\model\assets


Model path: runs:/56500be05c0443f08e7c8ed760d94110/model


In [35]:
import mlflow
from mlflow.keras import log_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Dropout
import time

# Inicia un nuevo run de MLflow
with mlflow.start_run(experiment_id=experiment_nn.experiment_id) as run:
    seed = 0
    mlflow.log_param('seed', seed)

    # Codifica las etiquetas a valores numéricos si aún no están codificadas
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
        
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=seed)

    # Define el modelo de la red neuronal con dropout
    model = Sequential()
    model.add(Dense(64, input_dim=10, activation='relu'))  # Capa de entrada con 10 variables de entrada
    model.add(Dropout(0.5))  # Capa de dropout
    model.add(Dense(32, activation='relu'))  # Capa oculta con 32 neuronas
    model.add(Dropout(0.5))  # Capa de dropout
    model.add(Dense(8, activation='softmax'))  # Capa de salida con 8 neuronas para las 8 categorías

    # Compila el modelo
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Entrena el modelo
    start_time = time.time()
    history = model.fit(X_train, y_train, epochs=200, batch_size=24, validation_data=(X_test, y_test))
    train_time = time.time() - start_time

    # Evalúa el modelo en el conjunto de prueba
    loss, accuracy = model.evaluate(X_test, y_test)
    print("Accuracy del modelo en el conjunto de prueba:", accuracy)
    
    # Log de los parámetros y métricas en MLflow
    mlflow.log_param('batch_size', 24)
    mlflow.log_param('epochs', 100)
    mlflow.log_metric('test_loss', loss)
    mlflow.log_metric('test_accuracy', accuracy)
    mlflow.log_metric('train_time', train_time)
    

    # Guarda el modelo en MLflow
    log_model(model, "model")
    print(f"Model path: runs:/{run.info.run_id}/model")


Epoch 1/200
4/4 [==============================] - 1s 59ms/step - loss: 861.7396 - accuracy: 0.1489 - val_loss: 335.3738 - val_accuracy: 0.0000e+00
Epoch 2/200
4/4 [==============================] - 0s 9ms/step - loss: 1051.3070 - accuracy: 0.1277 - val_loss: 249.0758 - val_accuracy: 0.0000e+00
Epoch 3/200
4/4 [==============================] - 0s 9ms/step - loss: 879.8327 - accuracy: 0.1277 - val_loss: 188.7435 - val_accuracy: 0.0000e+00
Epoch 4/200
4/4 [==============================] - 0s 9ms/step - loss: 854.7745 - accuracy: 0.1489 - val_loss: 175.3261 - val_accuracy: 0.0000e+00
Epoch 5/200
4/4 [==============================] - 0s 9ms/step - loss: 895.5030 - accuracy: 0.1915 - val_loss: 142.5095 - val_accuracy: 0.0000e+00
Epoch 6/200
4/4 [==============================] - 0s 9ms/step - loss: 706.7451 - accuracy: 0.1596 - val_loss: 119.4523 - val_accuracy: 0.0000e+00
Epoch 7/200
4/4 [==============================] - 0s 12ms/step - loss: 662.6778 - accuracy: 0.1702 - val_loss: 107.

2024/06/13 09:58:50 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


Accuracy del modelo en el conjunto de prueba: 0.4166666567325592
INFO:tensorflow:Assets written to: C:\Users\beca1\AppData\Local\Temp\tmpw5mdzcoe\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\beca1\AppData\Local\Temp\tmpw5mdzcoe\model\data\model\assets


Model path: runs:/8e6f7fea64de4b5285df0e1c03a1c57d/model


# SVM ⚙⚙⚙

In [36]:
experiment_svm = mlflow.create_experiment("svm")

In [37]:
experiment_svm

'760368007734881898'

In [38]:
with mlflow.start_run(experiment_id=experiment_svm) as run:
    seed = 0
    mlflow.log_param('seed', seed)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=seed)

    # Define los parámetros de búsqueda en rejilla
    param_grid = {
        'C': [0.01, 0.1, 1, 10],  # Prueba diferentes valores para el parámetro de regularización C
        'gamma': [0.0001, 0.001, 0.01, 0.1, 1],  # Prueba diferentes valores para el parámetro gamma del kernel
        'kernel': ['linear', 'rbf']  # Prueba diferentes tipos de kernel
    }

    mlflow.log_param('param_grid', param_grid)

    # Construye el modelo SVM
    svm = SVC(random_state=seed)

    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=seed)
    mlflow.log_param('cv', cv)

    # Realiza la búsqueda en rejilla
    grid_search = GridSearchCV(estimator=svm, param_grid=param_grid, cv=cv,
                               scoring='accuracy', n_jobs=-1)

    start_time = time.time()
    grid_result = grid_search.fit(X_train, y_train)
    train_time = time.time() - start_time

    # Muestra los mejores parámetros encontrados
    print("Mejores parámetros:", grid_result.best_params_)
    mlflow.log_param('best_params_', grid_result.best_params_)

    # Hacer predicciones en el conjunto de prueba utilizando el mejor modelo
    y_test_pred = grid_result.predict(X_test)

    # Calcular la precisión del modelo
    test_accuracy = accuracy_score(y_test, y_test_pred)
    print("Precisión del modelo en el conjunto de prueba:", test_accuracy)
    mlflow.log_metric('test_accuracy', test_accuracy)
    mlflow.log_metric('train_time', train_time)
    

    # Guarda el mejor modelo en MLflow
    mlflow.sklearn.log_model(grid_result.best_estimator_, "model")
    print(f"Model path: runs:/{run.info.run_id}/model")

C:\Users\beca1\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\beca1\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\beca1\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


Mejores parámetros: {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
Precisión del modelo en el conjunto de prueba: 0.7666666666666667
Model path: runs:/664502fd8e4c4ccfb2a8078827ef56f6/model
